# 01_03_adjust_mansion_unit_area outputs

Automated preview of deterministic pipeline outputs saved under `data/interim`. Update the `DATASETS` list if the step emits new files.


In [ ]:
from pathlib import Path

import pandas as pd
from IPython.display import display

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 400)

NOTEBOOK_RELATIVE_PATH = Path("notebooks/data/interim/01_03_adjust_mansion_unit_area/preview.ipynb")
try:
    NOTEBOOK_PATH = Path(__file__).resolve()
except NameError:  # running interactively
    NOTEBOOK_PATH = (Path.cwd() / NOTEBOOK_RELATIVE_PATH).resolve()


def resolve_project_root(notebook_path: Path) -> Path:
    for candidate in notebook_path.parents:
        if (candidate / "data").exists() and (candidate / "src").exists():
            return candidate
    return notebook_path.parents[-1]


PROJECT_ROOT = resolve_project_root(NOTEBOOK_PATH)
DATASETS = [
    ("train_kodate", PROJECT_ROOT / "data" / "interim" / "01_03_adjust_mansion_unit_area" / "train_kodate.parquet"),
    ("train_mansion", PROJECT_ROOT / "data" / "interim" / "01_03_adjust_mansion_unit_area" / "train_mansion.parquet"),
    ("test_kodate", PROJECT_ROOT / "data" / "interim" / "01_03_adjust_mansion_unit_area" / "test_kodate.parquet"),
    ("test_mansion", PROJECT_ROOT / "data" / "interim" / "01_03_adjust_mansion_unit_area" / "test_mansion.parquet"),
]
HEAD_ROWS = 20
COLUMN_OVERVIEW_SAMPLE_SIZE = 5
COLUMN_OVERVIEW_CHUNK_SIZE = 200
COLUMN_OVERVIEW_RANDOM_SEED = 314159


def build_column_overview(
    df: pd.DataFrame,
    *,
    sample_size: int = COLUMN_OVERVIEW_SAMPLE_SIZE,
    random_state: int | None = COLUMN_OVERVIEW_RANDOM_SEED,
) -> pd.DataFrame:
    row_count = len(df)
    overview_records: list[dict[str, object]] = []
    sample_columns = [f"random_sample_{i + 1}" for i in range(sample_size)]
    for idx, column in enumerate(df.columns):
        series = df[column]
        not_null_count = int(series.notna().sum())
        not_null_rate = (not_null_count / row_count * 100) if row_count else 0.0
        min_value = series.min(skipna=True)
        max_value = series.max(skipna=True)
        non_null = series.dropna()
        sample_n = min(sample_size, len(non_null))
        samples_list: list[object] = []
        if sample_n > 0:
            seed = None if random_state is None else random_state + idx
            samples_list = non_null.sample(n=sample_n, random_state=seed, replace=False).tolist()
        record: dict[str, object] = {
            "column_name": column,
            "row_count": row_count,
            "not_null_count": not_null_count,
            "not_null_rate_pct": round(not_null_rate, 2),
            "min": min_value,
            "max": max_value,
        }
        for sample_idx, col_name in enumerate(sample_columns):
            record[col_name] = samples_list[sample_idx] if sample_idx < len(samples_list) else pd.NA
        overview_records.append(record)
    return pd.DataFrame(overview_records)


def display_column_overview(
    df: pd.DataFrame | None,
    label: str,
    *,
    sample_size: int = COLUMN_OVERVIEW_SAMPLE_SIZE,
    chunk_size: int = COLUMN_OVERVIEW_CHUNK_SIZE,
    random_state: int | None = COLUMN_OVERVIEW_RANDOM_SEED,
) -> None:
    print(f"\n--- {label}: カラムサマリー ---")
    if df is None:
        print("⚠️ DataFrame が None のためサマリーを表示できません。")
        return
    if df.shape[1] == 0:
        print("(列が存在しません)")
        return
    overview = build_column_overview(
        df=df,
        sample_size=sample_size,
        random_state=random_state,
    )
    if overview.empty:
        print("(列が存在しません)")
        return
    total_cols = overview.shape[0]
    print(f"{total_cols} columns")
    for start in range(0, total_cols, chunk_size):
        end = min(start + chunk_size, total_cols)
        print(f"columns {start + 1}-{end} / {total_cols}")
        display(overview.iloc[start:end])


def inspect_dataset(label: str, path: Path, head_rows: int = HEAD_ROWS) -> None:
    if not path.exists():
        print(f"⚠️ Missing file: {path}")
        return
    df = pd.read_parquet(path)
    print(f"\n=== {label} ===")
    print(f"path: {path.relative_to(PROJECT_ROOT)}")
    print(f"shape: {df.shape[0]} rows x {df.shape[1]} cols")
    dtype_info = df.dtypes.astype(str)
    print("dtypes (first 20 columns):")
    print(dtype_info.head(20))
    if len(dtype_info) > 20:
        print("... (truncated)")
    sample = df.head(head_rows)
    display(sample.T)
    display_column_overview(df, label=f"{label} dataset")


In [ ]:
for label, path in DATASETS:
    inspect_dataset(label, path)



=== train_kodate ===
path: data/interim/01_03_adjust_mansion_unit_area/train_kodate.parquet
shape: 165310 rows x 139 cols
dtypes (first 20 columns):
data_id                       int64
target_ym                     int64
money_room                    int64
building_id                   int64
building_status               int64
building_create_date         string
building_modify_date         string
building_type                 int64
building_name                string
homes_building_name          string
homes_building_name_ruby     string
unit_count                  float64
full_address                 string
lon                         float64
lat                         float64
building_structure          float64
total_floor_area            float64
building_area               float64
floor_count                 float64
basement_floor_count        float64
dtype: object
... (truncated)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
data_id,0,1,2,3,5,6,7,8,9,10,11,13,14,15,18,19,20,21,22,23
target_ym,201901,201901,201901,201901,201901,201901,201901,201901,201901,201901,201901,201901,201901,201901,201901,201901,201901,201901,201901,201901
money_room,13980000,24480000,24480000,16300000,9000000,9900000,5400000,14500000,11680000,14800000,22800000,16000000,16800000,16500000,32000000,16000000,18800000,18000000,12980000,17800000
building_id,206271,83315,140201,216551,39357,88925,87743,175661,43281,136298,166012,61017,28221,167422,203963,47201,78176,112943,60223,170473
building_status,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
building_create_date,2014-06-27 21:09:41,2014-06-27 21:09:43,2014-06-27 21:09:43,2014-06-27 19:24:11,2014-06-27 19:24:12,2014-06-27 19:24:12,2014-06-27 21:09:52,2014-06-27 19:24:12,2014-06-27 19:24:12,2014-06-27 19:24:12,2014-06-27 19:24:14,2014-06-27 19:24:15,2014-06-27 19:24:16,2014-06-27 19:24:16,2014-06-27 19:24:17,2014-06-27 19:24:18,2014-06-27 19:24:18,2014-06-27 21:11:22,2014-06-27 21:11:24,2014-06-27 21:11:25
building_modify_date,2019-01-24 00:10:08,2019-04-03 00:10:08,2020-06-06 00:10:11,2019-04-26 00:10:08,2019-05-22 00:10:08,2020-10-26 00:10:05,2019-02-05 00:10:08,2019-02-06 00:10:08,2019-03-19 00:10:09,2019-05-26 00:10:09,2019-03-12 00:10:08,2022-06-01 00:10:05,2020-03-08 00:10:10,2019-01-19 00:10:09,2022-06-01 00:10:05,2020-12-01 00:10:05,2019-06-08 00:10:09,2025-03-11 00:45:08,2019-04-30 00:10:08,2019-02-05 00:10:08
building_type,4,4,4,4,4,4,999,4,4,999,4,4,4,4,4,4,999,4,4,4
building_name,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
homes_building_name,桑名市東正和台7丁目10-11,桑名市松ノ木5丁目,桑名市松ノ木七丁目,中古戸建 知多市八幡字荒井,知多市八幡字大平地,中古戸建 知多市八幡字小根,<NA>,中古戸建 知多市八幡字笹廻間,中古戸建 知多市八幡字笹廻間,<NA>,中古戸建 知多市八幡堀之内,中古戸建 知多市八幡新町3丁目,知多市西巽が丘一丁目,中古戸建 知多市西巽が丘1丁目,中古戸建 知多市長浦1丁目,知多市新知東町,<NA>,米原市入江中古戸建,米原市坂口 戸建て,米原市春照 中古住宅



--- train_kodate dataset: カラムサマリー ---
139 columns
columns 1-139 / 139


,column_name,row_count,not_null_count,not_null_rate_pct,min,max,random_sample_1,random_sample_2,random_sample_3,random_sample_4,random_sample_5
0,data_id,165310,165310,100.00,0,363922,214247,287780,81813,212193,309576
1,target_ym,165310,165310,100.00,201901,202207,202207,202201,202101,201907,201901
2,money_room,165310,165310,100.00,4900000,188000000,11200000,42800000,10000000,21000000,13800000
3,building_id,165310,165310,100.00,0,217793,128048,54594,100816,73572,1934
4,building_status,165310,165310,100.00,1,9,1,1,1,1,1
5,building_create_date,165310,165310,100.00,2014-06-27 19:12:07,2022-06-02 14:46:17,2014-06-27 22:13:50,2014-06-27 23:39:39,2014-06-27 20:24:05,2014-06-27 22:53:49,2014-06-28 02:38:01
6,building_modify_date,165310,165310,100.00,2019-01-02 00:10:08,2025-10-08 00:45:08,2021-09-05 00:10:05,2022-10-16 00:10:05,2019-12-09 11:21:51,2021-10-24 00:10:05,2022-06-04 00:10:05
7,building_type,165310,165310,100.00,1,999,4,4,4,4,4
8,building_name,165310,476,0.29,(有)サンキミツギ寮,黄介と茶々のアートハウス,プロスペラスタウンハウス,共栄産業(株)中村寮,医療法人仁寿会,アパート,デンタープレイス佐倉A棟
9,homes_building_name,165310,143555,86.84,おしゃれカッコいい室内 ・中茶屋1丁目 中古戸建,～～～鴻巣市境中古戸建て～土地面積90坪～,上尾市向山2丁目,田中南西浦町戸建,東大和市清水6丁目 戸建,新居町 中古戸建,塩竈市新富町 中古 4LDK納戸



=== train_mansion ===
path: data/interim/01_03_adjust_mansion_unit_area/train_mansion.parquet
shape: 198614 rows x 140 cols
dtypes (first 20 columns):
data_id                       int64
target_ym                     int64
money_room                    int64
building_id                   int64
building_status               int64
building_create_date         string
building_modify_date         string
building_type                 int64
building_name                string
homes_building_name          string
homes_building_name_ruby     string
unit_count                  float64
full_address                 string
lon                         float64
lat                         float64
building_structure          float64
total_floor_area            float64
building_area               float64
floor_count                 float64
basement_floor_count        float64
dtype: object
... (truncated)


...

,column_name,row_count,not_null_count,not_null_rate_pct,min,max,random_sample_1,random_sample_2,random_sample_3,random_sample_4,random_sample_5
0,data_id,52892,52892,100.00,1,99999,7987,93416,111517,18416,51128
1,id,52892,52892,100.00,1,112434,63724,36121,24080,4005,6773
2,target_ym,52892,52892,100.00,202301,202307,202307,202301,202301,202301,202301
3,building_id,52892,52892,100.00,0,217795,151936,177752,136017,120938,181892
4,building_status,52892,52892,100.00,1,9,1,1,1,1,1
5,building_create_date,52892,52892,100.00,2014-06-27 19:12:07,2023-06-05 17:16:41,2014-06-27 21:31:23,2018-08-02 19:24:20,2014-06-27 21:05:46,2014-06-27 21:46:05,2014-06-28 02:20:26
6,building_modify_date,52892,52892,100.00,2019-08-03 00:10:09,2025-10-08 00:45:08,2024-03-02 10:32:36,2023-03-20 00:10:05,2023-02-21 00:10:05,2024-01-09 00:10:05,2024-07-10 00:45:08
7,building_type,52892,52892,100.00,1,999,4,4,5,4,4
8,building_name,52892,87,0.16,(有)桑商社宅,風見ハイツ,グリーンヒル,みずでんビル,姫路耐火工業社員寮,Kフラット,ほのぼのハウス田能
9,homes_building_name,52892,46325,87.58,Re:Home 厚木市下川入 中古一戸建て,～駐車3台可～山科区勧修寺下ノ茶屋町,春日部市小渕 中古一戸建て,北区緑町6丁目,東鴻池町5-9-33,上京区長門町 中古戸建,梅ケ畑清水町



=== test_mansion ===
path: data/interim/01_03_adjust_mansion_unit_area/test_mansion.parquet
shape: 59545 rows x 140 cols
dtypes (first 20 columns):
data_id                      string
id                            int64
target_ym                     int64
building_id                   int64
building_status               int64
building_create_date         string
building_modify_date         string
building_type                 int64
building_name                string
homes_building_name          string
homes_building_name_ruby     string
unit_count                  float64
full_address                 string
lon                         float64
lat                         float64
building_structure          float64
total_floor_area            float64
building_area               float64
floor_count                 float64
basement_floor_count        float64
dtype: object
... (truncated)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
data_id,0,7,17,18,40,51,52,110,111,119,121,122,123,125,127,128,136,137,138,139
id,0,7,17,18,40,51,52,110,111,119,121,122,123,125,127,128,136,137,138,139
target_ym,202301,202301,202301,202301,202301,202301,202301,202301,202301,202301,202301,202301,202301,202301,202301,202301,202301,202301,202301,202301
building_id,129053,179871,123056,200406,34018,175709,175709,95812,67832,206676,186249,169224,27223,55976,4713,191715,119307,119307,119307,38618
building_status,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
building_create_date,2014-06-27 21:09:41,2014-06-27 21:09:54,2014-06-27 21:11:25,2014-06-27 21:11:26,2014-06-27 21:12:09,2014-06-27 21:12:28,2014-06-27 21:12:28,2014-06-27 21:13:07,2014-06-27 21:13:07,2014-06-27 19:24:31,2014-06-27 21:13:10,2014-06-27 21:13:11,2014-06-27 21:13:12,2014-06-27 21:13:12,2014-06-27 21:13:13,2014-06-27 19:24:31,2014-06-27 21:13:17,2014-06-27 21:13:17,2014-06-27 21:13:17,2014-06-27 21:13:17
building_modify_date,2025-10-02 00:45:07,2025-09-15 02:51:02,2025-09-23 00:45:08,2025-09-15 02:52:27,2025-09-15 01:17:12,2025-09-15 02:20:24,2025-09-15 02:20:24,2025-09-22 04:00:02,2025-09-30 00:45:07,2025-09-15 03:37:12,2025-09-15 03:04:42,2025-09-22 04:00:02,2025-09-15 03:04:42,2025-10-07 00:24:02,2025-09-15 03:04:42,2025-09-15 03:37:12,2025-09-19 15:58:46,2025-09-19 15:58:46,2025-09-19 15:58:46,2025-09-15 03:04:42
building_type,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
building_name,スペリア桑名三番館,ラドーニ長島風の館,エネッツ米原,サンビレッジ米原,シティーナ神立チェリー館C棟,ネオステージあきるの,ネオステージあきるの,タワーズランド北野田サウスタワー,タワーズランド北野田セントラルタワー,オリエントハイツ前原,北野田ロイヤルビブレ,宮路池住宅4号棟,ルネ白鷺,シティパーク北野田,メイツ北野田,ロイヤルマンション周船寺西,コスモ初芝,コスモ初芝,コスモ初芝,クローバーハイツ初芝ヴィプレ
homes_building_name,スペリア桑名三番館,ラドーニ長島 風の館,エネッツ米原 1階,サンビレッジ米原 3階,シティーナ神立チェリー館(C棟),ネオステージあきるの駅前通り,ネオステージあきるの駅前通り,タワーズランド北野田サウスタワー,タワーズランド北野田セントラルタワー,オリエントハイツ前原,ロイヤルビブレ北野田,宮路池住宅4号棟,ルネ白鷺,シティパーク北野田,メイツ北野田,ロイヤルマンション周船寺西,コスモ初芝,コスモ初芝,コスモ初芝,クローバーハイツ初芝ヴィプレ



--- test_mansion dataset: カラムサマリー ---
140 columns
columns 1-140 / 140


,column_name,row_count,not_null_count,not_null_rate_pct,min,max,random_sample_1,random_sample_2,random_sample_3,random_sample_4,random_sample_5
0,data_id,59545,59545,100.00,0,99995,40305,31343,87074,59717,7485
1,id,59545,59545,100.00,0,112436,69102,1181,100930,105874,64810
2,target_ym,59545,59545,100.00,202301,202307,202301,202301,202307,202307,202307
3,building_id,59545,59545,100.00,6,217796,12911,200163,10530,35676,210665
4,building_status,59545,59545,100.00,1,9,1,1,1,1,1
5,building_create_date,59545,59545,100.00,2014-06-27 19:12:07,2023-06-05 17:01:00,2014-06-28 02:23:07,2014-06-27 22:14:12,2014-06-28 00:41:45,2014-06-27 22:42:36,2014-06-27 23:48:18
6,building_modify_date,59545,59545,100.00,2023-01-13 00:10:05,2025-10-08 03:02:17,2025-09-15 02:03:55,2025-09-15 03:04:37,2025-09-22 04:00:02,2025-09-15 01:29:22,2025-09-15 01:21:55
7,building_type,59545,59545,100.00,1,999,1,1,1,1,1
8,building_name,59545,59544,100.00,'90ninety-1熊谷,麻生台団地26号棟,一光大井ハイツ高蔵西館,東川口宝マンション,光栄ハイツ,パレ・ロワイヤル田寺東ウエストコート,馬込マンション
9,homes_building_name,59545,58568,98.36,サカエ池袋マンション,～高層階・角部屋～ ザ・レジデンス宇都宮,ライオンズヒルズ豊田,アクアリーナ川崎アズリーノタワー,グリーンハイツ住吉,ロイヤルパレス長町南セントラス,注目の改装済中古マンション「志紀」駅から8分
